In [1]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [2]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x[batch_idx], y[batch_idx]
        yield x_batch, y_batch

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [4]:
lr = 1e-2
epoch = 20
batch_size = 200
clip_by_norm_constant = 1.0

In [5]:
n_inputs = 28 * 28  
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [6]:
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # there is no regularization loss term
    return max_norm

max_norm_reg = max_norm_regularizer(threshold=clip_by_norm_constant)

In [7]:
max_norm_reg

<function __main__.max_norm_regularizer.<locals>.max_norm(weights)>

In [8]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, n_hidden1, activation=tf.nn.relu, 
                              kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden1, activation=tf.nn.relu, 
                              kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name="entropy_loss")

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    opt = tf.train.GradientDescentOptimizer(lr)
    train = opt.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
with tf.Session() as sess:
    init.run()
    for i in range(epoch):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train, feed_dict={x: x_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})
        print(i, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./save/my_model_final.ckpt")

0 Validation accuracy: 0.8184
1 Validation accuracy: 0.8704
2 Validation accuracy: 0.8886
3 Validation accuracy: 0.9018
4 Validation accuracy: 0.9078
5 Validation accuracy: 0.913
6 Validation accuracy: 0.9178
7 Validation accuracy: 0.9214
8 Validation accuracy: 0.9236
9 Validation accuracy: 0.9296
10 Validation accuracy: 0.9312
11 Validation accuracy: 0.932
12 Validation accuracy: 0.934
13 Validation accuracy: 0.9364
14 Validation accuracy: 0.9388
15 Validation accuracy: 0.9404
16 Validation accuracy: 0.94
17 Validation accuracy: 0.9408
18 Validation accuracy: 0.9428
19 Validation accuracy: 0.9442
